# 1. Examine Model Size

parameters only

In [1]:
import matplotlib 
import subprocess
import torch
import pickle as pkl

# Load some common jupyter code
%run distiller_jupyter_helpers.ipynb
import ipywidgets as widgets
from ipywidgets import interactive, interact, Layout
from models import create_model
from apputils import *

In [2]:
ckpt_files = {
    'Original': '../resnet20_cifar_baseline/best.pth.tar',
    'Pruning_filter-wise': '../examples/classifier_compression/logs/2018.11.03-231903/best.pth.tar',
    'Pruning_element-wise': '../examples/classifier_compression/logs/2018.11.03-215240/best.pth.tar',
    'Quantization': '../examples/classifier_compression/logs/2018.11.04-145420/best.pth.tar'
}

model_dict = {
    'Original': create_model(False, 'cifar10', 'resnet20_cifar'),
    'Pruning_filter-wise': create_model(False, 'cifar10', 'resnet20_cifar'),
    'Pruning_element-wise': create_model(False, 'cifar10', 'resnet20_cifar'),
    'Quantization': create_model(False, 'cifar10', 'resnet20_cifar')
}

In [3]:
!rm *.pt

In [4]:
for k in ckpt_files:
    load_checkpoint(model_dict[k], ckpt_files[k]);
    torch.save(model_dict[k].module, '{}.pt'.format(k))
    #model_dict[k].module.save_state_dict('{}.pt'.format(k))

In [5]:
checkpoint = torch.load('../examples/classifier_compression/logs/2018.11.04-145420/best.pth.tar')
qmd = checkpoint['quantizer_metadata']
print(qmd['type'])

<class 'distiller.quantization.clipped_linear.DorefaQuantizer'>


In [6]:
model = model_dict['Quantization']

In [17]:
[k for k in model.state_dict().keys() if 'float' in k]

['module.layer1.0.conv1.float_weight',
 'module.layer1.0.conv2.float_weight',
 'module.layer1.1.conv1.float_weight',
 'module.layer1.1.conv2.float_weight',
 'module.layer1.2.conv1.float_weight',
 'module.layer1.2.conv2.float_weight',
 'module.layer2.0.conv1.float_weight',
 'module.layer2.0.conv2.float_weight',
 'module.layer2.0.downsample.0.float_weight',
 'module.layer2.1.conv1.float_weight',
 'module.layer2.1.conv2.float_weight',
 'module.layer2.2.conv1.float_weight',
 'module.layer2.2.conv2.float_weight',
 'module.layer3.0.conv1.float_weight',
 'module.layer3.0.conv2.float_weight',
 'module.layer3.0.downsample.0.float_weight',
 'module.layer3.1.conv1.float_weight',
 'module.layer3.1.conv2.float_weight',
 'module.layer3.2.conv1.float_weight',
 'module.layer3.2.conv2.float_weight']

In [7]:
weights = [model.state_dict()[k] for k in model.state_dict() if ('tracked' not in k) and \
                                                     ('float' not in k) and ('running' not in k)]
with open('Quantization.pt', 'wb') as f:
    pkl.dump(weights, f)

In [8]:
!ls -lah --block-size=K | grep .pt

-rw-r--r--  1 dllab users 1117K Nov  4 17:44 Original.pt
-rw-r--r--  1 dllab users  904K Nov  4 17:44 Pruning_element-wise.pt
-rw-r--r--  1 dllab users  803K Nov  4 17:44 Pruning_filter-wise.pt
-rw-r--r--  1 dllab users 1084K Nov  4 17:45 Quantization.pt


Original MACs/sparsity

In [23]:
!python ../examples/classifier_compression/compress_classifier.py \
                --arch resnet20_cifar /tmp/dataset-nctu -p=50 \
                --resume=../../resnet20_cifar_baseline/best.pth.tar \
                --summary sparsity

Log file for this run: /home/dllab/distiller/jupyter/logs/2018.11.04-082247/2018.11.04-082247.log
==> using cifar10 dataset
=> creating resnet20_cifar model for CIFAR10

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> loading checkpoint ../../resnet20_cifar_baseline/best.pth.tar
Checkpoint keys:
epoch
	arch
	state_dict
	best_top1
	optimizer
	compression_sched
   best top@1: 91.530
Loaded compression schedule from checkpoint (epoch 299)
=> loaded checkpoint '../../resnet20_cifar_baseline/best.pth.tar' (epoch 299)
Optimizer Type: <class 'torch.optim.sgd.SGD'>
Optimizer Args: {'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.0001, 'nesterov': False}

Parameters:
+----+-------------------------------------+----------------+---------------+----------------+------------+------------+----------+----------+----------+------------+---------+----------+------------+
|    | Name     

In [24]:
!python ../examples/classifier_compression/compress_classifier.py \
                --arch resnet20_cifar /tmp/dataset-nctu -p=50 \
                --resume=../../resnet20_cifar_baseline/best.pth.tar \
                --summary compute

Log file for this run: /home/dllab/distiller/jupyter/logs/2018.11.04-082255/2018.11.04-082255.log
==> using cifar10 dataset
=> creating resnet20_cifar model for CIFAR10

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> loading checkpoint ../../resnet20_cifar_baseline/best.pth.tar
Checkpoint keys:
epoch
	arch
	state_dict
	best_top1
	optimizer
	compression_sched
   best top@1: 91.530
Loaded compression schedule from checkpoint (epoch 299)
=> loaded checkpoint '../../resnet20_cifar_baseline/best.pth.tar' (epoch 299)
Optimizer Type: <class 'torch.optim.sgd.SGD'>
Optimizer Args: {'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.0001, 'nesterov': False}
+----+-----------------------+--------+----------+-----------------+--------------+-----------------+--------------+------------------+---------+
|    | Name                  | Type   | Attrs    | IFM             |   IFM volume | 

# 2. Examine Model Speed

In [12]:
!python ../examples/classifier_compression/compress_classifier.py \
                --arch resnet20_cifar /tmp/dataset-nctu -p=50 \
                --resume=../../resnet20_cifar_baseline/best.pth.tar \
                --evaluate

Log file for this run: /home/dllab/distiller/jupyter/logs/2018.11.04-174839/2018.11.04-174839.log
==> using cifar10 dataset
=> creating resnet20_cifar model for CIFAR10

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> loading checkpoint ../../resnet20_cifar_baseline/best.pth.tar
Checkpoint keys:
epoch
	arch
	state_dict
	best_top1
	optimizer
	compression_sched
   best top@1: 91.530
Loaded compression schedule from checkpoint (epoch 299)
=> loaded checkpoint '../../resnet20_cifar_baseline/best.pth.tar' (epoch 299)
Optimizer Type: <class 'torch.optim.sgd.SGD'>
Optimizer Args: {'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.0001, 'nesterov': False}
Files already downloaded and verified
Files already downloaded and verified
Dataset sizes:
	training=45000
	validation=5000
	test=10000
--- test ---------------------
10000 samples (256 per mini-batch)
==> Top1: 91.530    Top5: 99

In [13]:
!python ../examples/classifier_compression/compress_classifier.py \
                --arch resnet20_cifar /tmp/dataset-nctu -p=50 \
                --resume=../examples/classifier_compression/logs/2018.11.03-231903/best.pth.tar \
                --evaluate

Log file for this run: /home/dllab/distiller/jupyter/logs/2018.11.04-174850/2018.11.04-174850.log
==> using cifar10 dataset
=> creating resnet20_cifar model for CIFAR10

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> loading checkpoint ../examples/classifier_compression/logs/2018.11.03-231903/best.pth.tar
Checkpoint keys:
epoch
	arch
	state_dict
	best_top1
	optimizer
	compression_sched
	thinning_recipes
   best top@1: 90.220
Loaded compression schedule from checkpoint (epoch 434)
Loaded a thinning recipe from the checkpoint
Executed 1 recipes
=> loaded checkpoint '../examples/classifier_compression/logs/2018.11.03-231903/best.pth.tar' (epoch 434)
Optimizer Type: <class 'torch.optim.sgd.SGD'>
Optimizer Args: {'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.0001, 'nesterov': False}
Files already downloaded and verified
Files already downloaded and verified
Dataset sizes:
	

In [14]:
!python ../examples/classifier_compression/compress_classifier.py \
                --arch resnet20_cifar /tmp/dataset-nctu -p=50 \
                --resume=../examples/classifier_compression/logs/2018.11.03-215240/best.pth.tar \
                --evaluate

Log file for this run: /home/dllab/distiller/jupyter/logs/2018.11.04-174901/2018.11.04-174901.log
==> using cifar10 dataset
=> creating resnet20_cifar model for CIFAR10

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> loading checkpoint ../examples/classifier_compression/logs/2018.11.03-215240/best.pth.tar
Checkpoint keys:
epoch
	arch
	state_dict
	best_top1
	optimizer
	compression_sched
	thinning_recipes
   best top@1: 90.160
Loaded compression schedule from checkpoint (epoch 405)
Loaded a thinning recipe from the checkpoint
Executed 1 recipes
=> loaded checkpoint '../examples/classifier_compression/logs/2018.11.03-215240/best.pth.tar' (epoch 405)
Optimizer Type: <class 'torch.optim.sgd.SGD'>
Optimizer Args: {'lr': 0.1, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0.0001, 'nesterov': False}
Files already downloaded and verified
Files already downloaded and verified
Dataset sizes:
	

In [15]:
!python ../examples/classifier_compression/compress_classifier.py \
                --arch resnet20_cifar /tmp/dataset-nctu -p=50 \
                --resume=../examples/classifier_compression/logs/2018.11.04-145420/best.pth.tar \
                --evaluate

Log file for this run: /home/dllab/distiller/jupyter/logs/2018.11.04-174911/2018.11.04-174911.log
==> using cifar10 dataset
=> creating resnet20_cifar model for CIFAR10

--------------------------------------------------------
Logging to TensorBoard - remember to execute the server:
> tensorboard --logdir='./logs'

=> loading checkpoint ../examples/classifier_compression/logs/2018.11.04-145420/best.pth.tar
Checkpoint keys:
epoch
	arch
	state_dict
	best_top1
	optimizer
	compression_sched
	quantizer_metadata
   best top@1: 90.000
Loaded compression schedule from checkpoint (epoch 406)
Loaded quantizer metadata from the checkpoint
Preparing model for quantization using DorefaQuantizer
Parameter 'module.layer1.0.conv1.weight' will be quantized to 8 bits
Parameter 'module.layer1.0.conv2.weight' will be quantized to 8 bits
Parameter 'module.layer1.1.conv1.weight' will be quantized to 8 bits
Parameter 'module.layer1.1.conv2.weight' will be quantized to 8 bits
Parameter 'module.layer1.2.conv1.

Files already downloaded and verified
Files already downloaded and verified
Dataset sizes:
	training=45000
	validation=5000
	test=10000
--- test ---------------------
10000 samples (256 per mini-batch)
==> Top1: 89.990    Top5: 99.700    Loss: 0.330

--- 1.045027732849121 seconds ---

Log file for this run: /home/dllab/distiller/jupyter/logs/2018.11.04-174911/2018.11.04-174911.log
